In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup as BS
from datetime import datetime

import pandas as pd
import time
import sys, os
import re
import json, requests


def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)
def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)
def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)
def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

def find_name(name, browser):
    return browser.find_element(By.NAME, name)
def finds_name(name, browser):
    return browser.find_elements(By.NAME, name)

def find_tagName(tag_name, browser):
    return browser.find_element(By.TAG_NAME, tag_name)

def finds_tagName(tag_name, browser):
    return browser.find_elements(By.TAG_NAME, tag_name)

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    base_path = getattr(sys, '_MEIPASS', os.path.dirname(os.path.abspath(__file__)))
    return os.path.join(base_path, relative_path)

In [2]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-suage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path='chromedriver.exe')
    
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    return browser

In [3]:
def get_url(browser, url):
    browser.get(url)
    time.sleep(1)

In [4]:
df = pd.read_excel('전국_상장회사_data.xlsx', 'seoul_data')

In [5]:
url_list = df['홈페이지'].to_list()
company_name_list = df['회사명'].to_list()

In [6]:
print(len(url_list))
print(len(company_name_list))

1684
1684


In [7]:
url_list_test = url_list[1300:1401]
company_name_list_test = company_name_list[1300:1401]

In [ ]:
data = {}
count = 0

hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}


for url, company_name in zip(url_list, company_name_list):
    try:
        count2 = 0
        while True:
            response = requests.get(url, headers=hdr)
            count2 += 1
            if count2 == 10:
                break
            if response.status_code == 200:
                soup = BS(response.text, 'html.parser')
                if soup.find_all('html'):
                    page_text = soup.get_text()
                    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
                    email_formats = re.findall(email_pattern, page_text)
                    break

            else:
                print('Unable to access the homepage')
                break

        data[f'{company_name}'] = []
        for email in email_formats:
            data[f'{company_name}'].append({
                email
            })
        count += 1
        print(f'{count} : {company_name} / {email_formats}')
    except Exception as ex:
        pass

1 : 제이엔비 / ['jnb@jnbeng.com']
2 : 동인기연 / ['info@dong-in.com', 'info@dong-in.com']
3 : 에코아이 / ['info@dong-in.com', 'info@dong-in.com']
4 : 스톰테크 / ['bill@stormtec.co.kr', 'sales@stormtec.co.kr']
5 : 캡스톤파트너스 / ['E-MAILmaster@cspartners.co.kr', 'INVESTMENTbiz@cspartners.co.kr']
6 : 프로젠 / ['ir@progen.co.kr']
7 : 에이직랜드 / ['ir@progen.co.kr']
8 : 비아이매트릭스 / []
9 : 쏘닉스 / ['sawsales@sawnics.com']
10 : 세니젠 / []
11 : 유투바이오 / []
12 : 퀄리타스반도체 / ['sales@q-semi.com', 'recruit@q-semi.com', 'ir@q-semi.com']
13 : 미쥬 / []
14 : 워트 / ['wot8600@wot.co.kr']
15 : 에스엘에스바이오 / []
16 : 아이엠티 / ['inquiry.imt@gmail.com', 'sales_imt@imt-c.co.kr', 'inquiry.imt@gmail.com', 'sales_imt@imt-c.co.kr']
17 : 레뷰코퍼레이션 / ['help@revu.net']
18 : 두산로보틱스 / ['help@revu.net']
19 : 한싹 / []
20 : 밀리의서재 / []
21 : 인스웨이브시스템즈 / []
22 : 코어라인소프트 / ['Esales@corelinesoft.com', 'Eobd@corelinesoft.com', 'Esupport@corelinesoft.com', 'Eir@corelinesoft.com', 'Einfo@corelinesoft.eu', 'Fusa@corelinesoft.com']
23 : 율촌 / []
24 : 한화플러스제4호스팩 / []
25 : 크라우드웍

In [165]:
data

{'제이엔비': [{'jnb@jnbeng.com'}],
 '동인기연': [{'info@dong-in.com'}, {'info@dong-in.com'}],
 '에코아이': [{'info@dong-in.com'}, {'info@dong-in.com'}],
 '스톰테크': [{'bill@stormtec.co.kr'}, {'sales@stormtec.co.kr'}],
 '캡스톤파트너스': [{'E-MAILmaster@cspartners.co.kr'},
  {'INVESTMENTbiz@cspartners.co.kr'}],
 '프로젠': [{'ir@progen.co.kr'}],
 '에이직랜드': [{'ir@progen.co.kr'}],
 '비아이매트릭스': [],
 '쏘닉스': [{'sawsales@sawnics.com'}],
 '세니젠': [],
 '유투바이오': [],
 '퀄리타스반도체': [{'sales@q-semi.com'}, {'recruit@q-semi.com'}, {'ir@q-semi.com'}],
 '미쥬': [],
 '워트': [{'wot8600@wot.co.kr'}],
 '에스엘에스바이오': [],
 '아이엠티': [{'inquiry.imt@gmail.com'},
  {'sales_imt@imt-c.co.kr'},
  {'inquiry.imt@gmail.com'},
  {'sales_imt@imt-c.co.kr'}],
 '레뷰코퍼레이션': [{'help@revu.net'}],
 '두산로보틱스': [{'help@revu.net'}],
 '한싹': [],
 '밀리의서재': [],
 '인스웨이브시스템즈': [],
 '코어라인소프트': [{'Esales@corelinesoft.com'},
  {'Eobd@corelinesoft.com'},
  {'Ecsd@corelinesoft.com'},
  {'Eir@corelinesoft.com'},
  {'Einfo@corelinesoft.eu'},
  {'Fusa@corelinesoft.com'}],
 '율촌': []

In [171]:
rows = []
for company, emails in data.items():
    for email_set in emails:
        for email in email_set:
            rows.append({'company_name': company, 'Email': email})

In [167]:
final_df = pd.DataFrame(rows)

In [172]:
final_df = final_df.drop_duplicates(subset='company_name', keep='first')
print(final_df)

    company_name                          Email
0           제이엔비                 jnb@jnbeng.com
1           동인기연               info@dong-in.com
3           에코아이               info@dong-in.com
5           스톰테크            bill@stormtec.co.kr
7        캡스톤파트너스  E-MAILmaster@cspartners.co.kr
..           ...                            ...
748      오리엔트바이오     deokgyeom.kim@orient.co.kr
749         한신공영              gssa@hanshinc.com
755         신라교역            webmaster@sla.co.kr
756        GS글로벌             jgmoon@gsgcorp.com
759       태양금속공업       taeyang@taeyangmetal.com

[398 rows x 2 columns]


In [173]:
final_df.to_excel('test_email2.xlsx', index=False)

In [4]:
browser = open_browser()

In [ ]:
start = time.time()
hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}

url_list = []
count = 1
for i in range(11, 500+1):
    
    url = f'https://www.saramin.co.kr/zf_user/search?search_area=main&search_done=y&search_optional_item=n&loc_mcd=101000&recruitPage={i}&recruitSort=relation&recruitPageCount=100&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&searchword=&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=n'
    response = requests.get(url, headers=hdr)
    if response.status_code == 200:
        soup = BS(response.text, 'html.parser')
        for a_href in soup.find_all(class_='track_event data_layer'):
            url_list.append('https://www.saramin.co.kr'+a_href['href'])
    print(count)
    count += 1
            
url_list = list(set(url_list))
end = time.time()
print(f"{end - start:.5f} sec")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148


In [63]:
len(url_list)

650

In [79]:
start = time.time()
homepage_urls = []
company_names = []

count = 1

for url in url_list:
    response2 = requests.get(url, headers=hdr)
    
    if response2.status_code == 200:
        soup = BS(response2.text, 'html.parser')

        dt_elements = soup.find_all('dt', string='홈페이지')

        for dt_element in dt_elements:
            dd_element = dt_element.find_next_sibling('dd')
            if dd_element:
                a_tags = dd_element.find_all('a')
                for a_tag in a_tags:
                    homepage_urls.append(a_tag['href'])
                    company_name = soup.find('title').text.split(' ')[0]
                    company_names.append(company_name)
    print(count)
    count += 1
    
end = time.time()
print(f"{end - start:.5f} sec")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [80]:
len(homepage_urls)

456

In [81]:
data = {}
count = 0

hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}


for url,company_name in zip(homepage_urls, company_names):
    try:
        count2 = 0
        while True:
            response = requests.get(url, headers=hdr)
            count2 += 1
            if count2 == 10:
                break
            if response.status_code == 200:
                soup = BS(response.text, 'html.parser')
                if soup.find_all('html'):
                    page_text = soup.get_text()
                    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
                    email_formats = re.findall(email_pattern, page_text)
                    break

            else:
                print('Unable to access the homepage')
                break

        data[f'{company_name}'] = []
        for email in email_formats:
            data[f'{company_name}'].append({
                email
            })
        count += 1
        print(f'{count} : {company_name} / {email_formats}')
    except Exception as ex:
        pass

1 : (주)아이엔티랩 / ['chief@intlab.co.kr']
2 : (주)씨메스 / ['Sales@cmesrobotics.ai', 'marketing@cmesrobotics.ai', 'info@cmesrobotics.com']
3 : (주)극동엠이에스 / []
4 : 한양여자대학교산학협력단 / []
5 : (주)한건이엔씨 / ['creativehg@naver.comAddr']
6 : (주)이퀴닉스 / ['champhelp@hackers.com']
7 : (주)챔프스터디 / ['soonsoof@naver.com', 'soonsoof@naver.com']
8 : (주)순수식품 / []
9 : 현대건설(주) / []
10 : 세원정공(주) / ['lionkorea@lion-korea.com', 'lionkorea@lion-korea.com', 'lionkorea@lion-korea.com']
11 : (주)라이온코리아 / []
12 : 엔알에스한익스(주) / ['hmlk@hm-link.com', 'hmlk@hm-link.com']
13 : 휴먼링크 / []
14 : (주)명동실탄사격장 / []
15 : (주)에이치알비젼 / []
16 : (주)렛유인에듀 / []
17 : (주)교원위즈 / []
18 : (주)더블유엔파트너스 / ['biz@leecompany.kr', 'biz@leecompany.kr']
19 : 잼팟(주) / []
20 : (주)트래솔 / ['info@hri.co.kr']
21 : (주)리컴퍼니 / ['cs@topfield.kr']
22 : 엔컴(주) / ['Jbp.info@jbpartners.co.kr']
23 : (주)현대경제연구원 / []
24 : (주)월더 / []
25 : (주)제이비파트너스 / ['magicsplit.yes@gmail.com', 'wongi0346@splitinvest.co.kr', 'magicsplit.yes@gmail.com']
26 : 인탭 / ['online@ozkiz.com', 'id@individuel.k

222 : (사)한국금융연구원 / []
223 : (주)갤리언컨설팅그룹 / []
224 : 컴포즈커피신림1호점 / ['info@unnet.co.kr']
225 : 로마옥마곡점 / []
226 : HAPE헤이프 / ['info@jooam.co.kr']
227 : 피엠씨바이오제닉스코리아(주) / []
228 : (주)청송 / []
229 : (주)테소로 / []
230 : 에버브레인컨설팅(주) / []
231 : 궁말11 / []
232 : 맨파워프로 / []
233 : 고려세무법인 / ['business@triz-group.com', 'business@triz-group.com', 'business@triz-group.com']
234 : (주)제니엘 / ['eduplex@eduplex.net']
235 : (주)웨이코스 / []
236 : (주)투비파트너즈 / []
237 : (주)브랜뉴랩 / []
238 : 중앙일보(주) / []
239 : (주)샐러디 / []
240 : (주)모닝인 / []
241 : 하루플란트 / []
242 : 진광이엔지(주) / ['newkidson@newkidson.co.kr', 'khhwang@newkidson.co.kr']
243 : (의)케이씨엘의료재단 / []
244 : 올림피아코리아(주) / []
245 : (주)삼우마테크 / ['mz@mz.co.kr']
246 : 진영(주) / ['dsmoon@sllife.co.kr', 'dsmoon@sllife.co.kr']
247 : (주)세중통상 / []
248 : (주)잘론네츄럴 / []
Unable to access the homepage
249 : (주)더원테크 / []
250 : 라비니움(청람빌딩) / ['webmaster@beautifulstore.org']
251 : 캡슐미디어(주) / ['tech@somansa.com', 'sales@somansa.com']
252 : (주)윌앤비전 / ['camiane_info@camiane.com', 'camiane_info@cami

In [82]:
rows = []
for company, emails in data.items():
    for email_set in emails:
        for email in email_set:
            rows.append({'company_name': company, 'Email': email})
            
final_df = pd.DataFrame(rows)
final_df = final_df.drop_duplicates(subset='company_name', keep='first')
print(final_df)

    company_name                     Email
0       (주)아이엔티랩        chief@intlab.co.kr
1         (주)씨메스     Sales@cmesrobotics.ai
4       (주)한건이엔씨  creativehg@naver.comAddr
5        (주)이퀴닉스     champhelp@hackers.com
6       (주)챔프스터디        soonsoof@naver.com
..           ...                       ...
310  (주)맨파워그룹코리아           sd90@sd90.co.kr
312     보울박스한남본점       webmaster@t-ime.com
313      신성통상(주)     firstactors@naver.com
314        네오리소스     blue@bluemeditech.com
315     (주)정직한도움     blue@bluemeditech.com

[191 rows x 2 columns]


In [83]:
final_df.to_excel('test_email3.xlsx', index=False)

--------------

-------------

In [34]:
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
email_formats = re.findall(email_pattern, page_text)

for email in email_formats:
    print(email)

In [8]:
main = browser.window_handles

for i in main:
    if i != main[0]:
        browser.switch_to.window(i)
        browser.close()

jnb@jnbeng.com


In [15]:
soup = BS(browser.page_source, 'html.parser')

footer_text = soup.find('footer').get_text() if soup.find('footer') else None
print(footer_text)



(주)제이엔비 17708 경기도 평택시 진위면 진위2산단로 31-21
TEL. 031)8015-2001    FAX. 031)8015-2005    MAIL. jnb@jnbeng.com

COPYRIGHT 2021⒞ JNB, ALL RIGHTS RESERVED.




In [10]:
browser = open_browser()

for url in url_list:
    get_url(browser, url)
    soup = BS(browser.page_source, 'html.parser')

    footer_text = soup.find('footer').get_text() if soup.find('footer') else None
    print(footer_text)
    print('--' * 20)
    time.sleep(1)



(주)제이엔비 17708 경기도 평택시 진위면 진위2산단로 31-21
TEL. 031)8015-2001    FAX. 031)8015-2005    MAIL. jnb@jnbeng.com

COPYRIGHT 2021⒞ JNB, ALL RIGHTS RESERVED.


----------------------------------------


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.161)
Stacktrace:
	GetHandleVerifier [0x00007FF6B64A5E42+3538674]
	(No symbol) [0x00007FF6B60C4C02]
	(No symbol) [0x00007FF6B5F75AEB]
	(No symbol) [0x00007FF6B5F5288C]
	(No symbol) [0x00007FF6B5FE5DD7]
	(No symbol) [0x00007FF6B5FFB40F]
	(No symbol) [0x00007FF6B5FDEE53]
	(No symbol) [0x00007FF6B5FAF514]
	(No symbol) [0x00007FF6B5FB0631]
	GetHandleVerifier [0x00007FF6B64D6CAD+3738973]
	GetHandleVerifier [0x00007FF6B652C506+4089270]
	GetHandleVerifier [0x00007FF6B6524823+4057299]
	GetHandleVerifier [0x00007FF6B61F5C49+720121]
	(No symbol) [0x00007FF6B60D126F]
	(No symbol) [0x00007FF6B60CC304]
	(No symbol) [0x00007FF6B60CC432]
	(No symbol) [0x00007FF6B60BBD04]
	BaseThreadInitThunk [0x00007FFECD217344+20]
	RtlUserThreadStart [0x00007FFECE4426B1+33]


footer을 가져오는 것은 None가 너무 많이 발생해 Pass
다른 방안을 고안

In [92]:
get_url(browser ,url_list[3])

In [95]:
company_name_list[3]

'스톰테크'

In [14]:
soup = BS(browser.page_source, 'html.parser')

# "@"이 포함된 텍스트만 출력
# 정규식 패턴
email_pattern = re.compile(r'^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')
email_pattern2 = re.compile(r'^.*[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')

elements = soup.get_text()
elements = elements.split('\n')

data = {}


for i in elements:
    matches = re.findall(email_pattern2, i)
    if len(matches) >= 1:
        print(matches)

['TEL. 031)8015-2001\xa0\xa0\xa0\xa0FAX. 031)8015-2005\xa0\xa0\xa0\xa0MAIL. jnb@jnbeng.com']
['TEL. 031)8015-2001\xa0\xa0\xa0\xa0FAX. 031)8015-2005\xa0\xa0\xa0\xa0MAIL. jnb@jnbeng.com']


In [14]:
def add_email_info(company, email):
    try:
        with open('email_info.json', 'r', encoding='utf-8') as json_file:
            email_info = json.load(json_file)
    except FileNotFoundError:
        # 파일이 없는 경우 초기화
        email_info = {}

    # 기존 정보에 새로운 정보 추가
    email_info[company] = email

    # JSON 형식으로 저장
    with open('email_info.json', 'w', encoding='utf-8') as json_file:
        json.dump(email_info, json_file, ensure_ascii=False, indent=2)

    print(f'{company}의 이메일 정보가 추가되었습니다.')

In [ ]:
browser = open_browser()

for url, company_name in zip(url_list, company_name_list):
    try:
        get_url(browser, url)

        soup = BS(browser.page_source, 'html.parser')

        # "@"이 포함된 텍스트만 출력
        # 정규식 패턴
        email_pattern = re.compile(r'^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')
        email_pattern2 = re.compile(r'^.*[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')

        elements = soup.get_text()
        elements = elements.split('\n')

        data = {}


        for i in elements:
            matches = re.findall(email_pattern2, i)
            if len(matches) >= 1:
                email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
                emails = re.findall(email_pattern, matches[0])
                add_email_info(company, emails)
#                 print(f"{company_name}: {emails}")
    except:
        pass

In [12]:
matches = ['Robots for Every Workplace쉽고 경제적인 협동로봇으로 안전하고 편리한 자동화를 선도합니다.Neuromeka.협동로봇의 다양한 활용을 통해 제조공정의 생산성 향상, F&B 서비스의 안정성 확대 그리고 문화예술 산업의 새로운 가능성을 뉴로메카가 함께합니다.Robot.협동로봇과 델타로봇 등 다양한 로봇 라인업을 통해 생산성 향상과 원가 절감 효과를 기대할 수 있으며 동시에 산업재해 고민도 해결할 수 있습니다.Video.동영상 보기동영상 보기01:20동영상 보기동영상 보기02:17동영상 보기동영상 보기01:44동영상 보기동영상 보기01:19동영상 보기동영상 보기03:48동영상 보기동영상 보기02:08동영상 보기동영상 보기04:39Post.News.7일 전뉴로메카, SK네트웍스서비스와 로봇 공급 총판 계약..."기간은 2년, 이후엔 갱신"12월 13일지능로봇 리빙랩 기반 인재육성 나선다…수원시 협약11월 10일교촌 치킨을 뉴로메카 로봇이 만든다12345Robot as a Tool |\xa0Robot as a Service |\xa0Robots for Every Workplace\u200b(주)뉴로메카\xa0 \xa0 \xa0서울특별시 성동구 아차산로 78, 에코넷센터 (04782) \xa0 \xa0 TEL : 1661-0773\xa0 \xa0 \xa0영업 : sales@neuromeka.com\xa0 \xa0 \xa0마케팅/홍보 : pr@neuromeka.com']

In [13]:
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
emails = re.findall(email_pattern, matches[0])
print(emails)

['sales@neuromeka.com', 'pr@neuromeka.com']


In [10]:
email_addresses = []

for a_tag in soup.find_all('a', href=True):
    if 'mailto:' in a_tag['href']:
        email_addresses.append(a_tag['href'].split(':')[1])

for span_tag in soup.find_all('span'):
    email_addresses.append(span_tag.get_text())

# 결과 출력
# print(email_addresses)
for i in email_addresses:
    matches = re.findall(email_pattern, i)
    print(matches)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [18]:
from bs4 import BeautifulSoup

# 예시로 사용할 HTML
html = """
<html>
  <body>
    <p>This is the first paragraph.</p>
    <p>This is the second paragraph.</p>
    <p>This is the third paragraph.</p>
  </body>
</html>
"""

# BeautifulSoup 객체 생성
soup = BeautifulSoup(html, 'html.parser')

# 모든 텍스트 가져오기
all_text = soup.get_text()

all_text.split('\n')

['',
 '',
 '',
 'This is the first paragraph.',
 'This is the second paragraph.',
 'This is the third paragraph.',
 '',
 '',
 '']